### Imports

In [ ]:
# import necessary functions and libraries
from helpers.readers import Reader
from helpers.visualization import Visualization3D
from sklearn.cluster import DBSCAN 
import open3d as o3d
import numpy as np 
import pandas as pd

### Read desired file

In [ ]:
# read data from
reader = Reader('/home/dim26fa/data/imod_models/mitochondria/train/')
reader.get_folders()

In [ ]:
reader.set_folder(2)

In [ ]:
reader.get_files_from_folder()

In [ ]:
reader.set_file(0)

In [ ]:
reader.read_txt(columns=4)

In [ ]:
reader.extract_xyz(column_names=[0,1,2])

In [ ]:
#reader.file = '/home/dim26fa/data/ries_lab_data/201'
reader.data = pd.read_csv('/home/dim26fa/data/imod_models/mitochondria/train/sample_0/mitochondriaLocalizations.txt')
data_xyz = reader.extract_xyz(column_names=['Pos_x','Pos_y','Pos_z'])

In [ ]:
viz = Visualization3D(reader.df_xyz).get_3d_scatter(color='blue').show()

In [ ]:
reader.file.split(sep='.')

In [ ]:
xyz = data_xyz.to_numpy()

### Point cloud downsampling

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [ ]:
pcd

In [ ]:
labels = np.array(pcd.cluster_dbscan(eps=1, min_points=50))

In [ ]:
len(labels)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label 
if max_label > 0 else 1))

In [ ]:
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [ ]:
pcd

In [ ]:
down_pcd = pcd.voxel_down_sample(voxel_size=100)
downpc_array = pd.DataFrame(np.asarray(down_pcd.points), columns = ['x','y','z'])

In [ ]:
o3d.visualization.draw_geometries([down_pcd])

In [ ]:
o3d_pc = o3d.io.read_point_cloud(reader.file, format = 'xyz')

In [ ]:
downpc = pcd.uniform_down_sample(100)
downpc_array = pd.DataFrame(np.asarray(downpc.points), columns = ['x','y','z'])

In [ ]:
downpc

In [ ]:
reader.file

In [ ]:
o3d_pc = o3d.io.read_point_cloud(reader.file.split(sep='.')[0]+'.txt', format='xyz')
downpc = o3d_pc.uniform_down_sample(100)
downpc_array = pd.DataFrame(np.asarray(downpc.points), columns = ['x [nm]','y [nm]','z [nm]'])

In [ ]:
o3d_pc

In [ ]:
viz2 = Visualization3D(downpc_array).get_3d_scatter(size=1).show()

### Find optimal eps for DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt 

nbrs = NearestNeighbors(n_neighbors=20).fit(data_xyz)
distances, indices = nbrs.kneighbors(downpc_array)
distances = distances[:,2]
distances = np.sort(distances, axis=0)
plt.plot(distances)

### Try out HDBSCAN for denoising

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=40, min_samples=None, algorithm='best', alpha=0.7,metric='euclidean')
clusterer.fit(data_xyz)

In [ ]:
clusterer.labels_

In [ ]:
len(clusterer.labels_)

In [ ]:
type(clusterer.labels_)

In [ ]:
np.unique(clusterer.labels_)

In [ ]:
import seaborn as sns

In [ ]:
threshold = pd.Series(clusterer.outlier_scores_).quantile(0.8)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]

In [ ]:
color_palette = sns.color_palette('deep', 3508)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
vis = Visualization3D(data_xyz).get_3d_scatter(color = cluster_colors, size = 1).show()

### Extract clusters

In [ ]:
clusterer.labels_[200]

In [ ]:
itemindex = np.where(clusterer.labels_ == clusterer.labels_[1])

In [ ]:
np.unique(clusterer.labels_)

In [ ]:
cluster = data_xyz.iloc[itemindex]

In [ ]:
type(clusterer.labels_s_)

In [ ]:
data_xyz.insert(3,"label",clusterer.labels_)

In [ ]:
data_xyz.to_csv('/home/dim26fa/data/ries_lab_data/labeled/labeled_201008_U2OS-Nup96-SNAP-AF647-anti-Elys-CF660C_cell3_sml.csv')

In [ ]:
cluster

In [ ]:
path = '/home/dim26fa/data/ries_lab_data/extracted_clusters/hdbscan40_201008_U2OS-Nup96-SNAP-AF647-anti-Elys-CF660C_cell1_sml/'

In [ ]:
for label in clusterer.labels_:
    indices = np.where(clusterer.labels_ == label)
    cluster = data_xyz.iloc[indices]
    cluster.to_csv(path + str(label) + '.csv', index=False)

In [ ]:
reader.data

In [ ]:
reader.data = pd.read_csv('/home/dim26fa/data/ries_lab_data/extracted_clusters/hdbscan40_201008_U2OS-Nup96-SNAP-AF647-anti-Elys-CF660C_cell1_sml/1356.csv')
data_xyz_cluster = reader.extract_xyz(column_names=['x','y','z'])

In [ ]:
reader.data

In [ ]:
vis_cluster = Visualization3D(data_xyz_cluster).get_3d_scatter(size=2, color='red').show()

### Denoise point cloud

In [ ]:
outliers

In [ ]:
data_without_outliers = downpc_array.drop(downpc_array.index[outliers])

In [ ]:
viz4 = Visualization3D(data_without_outliers).get_3d_scatter(size=2, color='red').show()

### Construct graphs 

In [ ]:
from sklearn.neighbors import KNeighborsTransformer
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

In [ ]:
pc_size = len(data_without_outliers)

In [ ]:
transformer = KNeighborsTransformer(n_neighbors = 25, algorithm='kd_tree')
transformer.fit_transform(data_xyz_cluster)

In [ ]:
graph = transformer.kneighbors_graph()
nx_graph = nx.from_numpy_array(graph.toarray())
len(nx_graph.edges)

In [ ]:
plt.imshow(graph.toarray(), cmap='jet')

In [ ]:
net = Network(notebook = True)
net.show_buttons(filter_=['physics'])
net.from_nx(nx_graph)
net.show('graph.html')

In [ ]:
A=list(nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph))

In [ ]:
net = Network(notebook = True)
net.show_buttons(filter_=['physics'])
net.from_nx(A[0])
net.show('graph.html')